In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# ----------------------------
# Load MNIST Dataset
# ----------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Flatten
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# One-hot labels (for categorical_crossentropy)
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# ----------------------------
# Model Builder
# ----------------------------
def build_model(activation, loss_fn):
    model = Sequential([
        Dense(256, activation=activation, input_shape=(784,)),
        Dense(128, activation=activation),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss=loss_fn,
        metrics=['accuracy']
    )
    return model

# ----------------------------
# Experiments
# ----------------------------
activations = ['relu', 'tanh', 'sigmoid']
loss_functions = ['sparse_categorical_crossentropy', 'categorical_crossentropy']

results = {}

for act in activations:
    for loss_fn in loss_functions:
        print(f"\nTraining with activation={act}, loss={loss_fn}")

        model = build_model(act, loss_fn)

        if loss_fn == 'categorical_crossentropy':
            history = model.fit(
                x_train, y_train_cat,
                validation_data=(x_test, y_test_cat),
                epochs=10,
                batch_size=128,
                verbose=0
            )
        else:
            history = model.fit(
                x_train, y_train,
                validation_data=(x_test, y_test),
                epochs=10,
                batch_size=128,
                verbose=0
            )

        test_loss, test_acc = model.evaluate(
            x_test,
            y_test_cat if loss_fn == 'categorical_crossentropy' else y_test,
            verbose=0
        )

        results[(act, loss_fn)] = test_acc
        print(f"Test Accuracy: {test_acc:.4f}")

# ----------------------------
# Display Results
# ----------------------------
print("\nSummary of Results:")
for k, v in results.items():
    print(f"Activation: {k[0]:8s} | Loss: {k[1]:35s} | Accuracy: {v:.4f}")